# Conformal Field Theory data

This notebook contains classes representing objects in CFTs:
* the central charge
* conformal dimensions
* fields

In [2]:
#when not working on this notebook set to false to avoid running cells when importing
run_tests=true;

#packages
using BenchmarkTools

## Central charge

The central charge $c$ can be parametrized by the variables $B$, $b$ or $\beta$ such that
$$
c = 13 + 6B + 6 B^{-1} \quad , \quad B = b^2 = -\beta^2 
$$
Conversely, we have 
$$
B = \frac{c-13 \pm \sqrt{(c-1)(c-25)}}{12}
$$

In [12]:
#= The central charge can be given directly or as a function of b,β,B. 
    No matter what parameter is passed we compute B from it, then b,β,c. 
    We ensure that the relation between b and beta never changes (sqrt branch). =#

Bfrom(parameter,value) = Dict(
        "c" => (value-13+sqrt(Complex{BigFloat}((value-1)*(value-25))))/12,
        "b" => value^2,
        "β" => -value^2,
        "B" => value
    )[parameter]

Bto(parameter,value) = Dict(
        "c" => 13+6*value+6/value,
        "b" => sqrt(Complex{BigFloat}(value)),
        "β" => im*sqrt(Complex{BigFloat}(value)),
        "B" => value
    )[parameter]

struct CentralCharge

    values::Dict{String,Complex{BigFloat}}

    function CentralCharge(parameter="c",value=1)
        B=Bfrom(parameter,value)
        dict=Dict(key => Bto(key,B) for key in ("c","b","β","B"))
        new(dict)
    end

end

#Print an object of type CentralCharge
function Base.show(io::IO,charge::CentralCharge)
    for (key,value) in charge.values
        println(io, "$key = $value")
    end
end

#overload [] to access values in charge
Base.getindex(charge::CentralCharge,key) = charge.values[key]

In [13]:
if run_tests
    charge=CentralCharge("c",0.5)
    println(charge)
    println("b = ",charge["b"])
end

B = -0.75 + 0.0im
c = 0.5 + 0.0im
b = 0.0 + 0.8660254037844386467637231707529361834714026269051903140279034897259665084543988im
β = -0.8660254037844386467637231707529361834714026269051903140279034897259665084543988 + 0.0im

b = 0.0 + 0.8660254037844386467637231707529361834714026269051903140279034897259665084543988im


## Conformal weights

Non-logarithmic fields in 2D CFTs are eigenvectors of the generators $L_0$ and $\bar L_0$ of the Virasoro algebra, with eigenvalues called conformal weights or conformal dimensions and respectively denoted $(\Delta,\bar \Delta)$. Conformal weights also make sense for logarithmic fields, though not as eigenvalues of $L_0,\bar L_0$, see [arXiv:2007.04190](https://arxiv.org/abs/2007.04190).

Conformal weights are conveniently parametrised in terms of the momentum $P$, parameters $\delta$ or $p$:

$$
\Delta = \frac{c-1}{24} + \delta  \quad , \quad \delta = -P^2 = p^2
$$

The Kac parametrisation for conformal weights is
$$ p_{(r,s)}=\frac{1}{2}(b r - b^{-1}s)$$

where $r,s$ are arbitrary numbers. We say the field is degenerate if $r,s\in \mathbb Z$ and rs>0. In terms of $r,s$, the dimension $\Delta$ is written


$$\Delta_{(r,s)} = \frac14 B (1-r^2) + \frac12 (1-rs) + \frac14\frac{1-s^2}{B}
$$

This convention is consistent with the one in in [this code](https://gitlab.com/s.g.ribault/Bootstrap_Virasoro.git)  but differs from the one in [this paper](https://arxiv.org/abs/2208.14298); it should be understood that the $P_{(r,s)}$ in the paper are the $p_{(r,s)}$ in this code.

In [14]:
p_from(parameter,value,charge::CentralCharge) = Dict(
        "Δ" => sqrt(Complex{BigFloat}(value - (charge["c"]-1)/24)),
        "δ" => sqrt(Complex{BigFloat}(value)),
        "P" => Complex{BigFloat}(-im*value),
        "p" => Complex{BigFloat}(value)
    )[parameter]

p_to(parameter,value,charge::CentralCharge) = Dict(
        "Δ" => Complex{BigFloat}(value^2 + (charge["c"]-1)/24),
        "δ" => Complex{BigFloat}(value^2),
        "P" => Complex{BigFloat}(im*value),
        "p" => Complex{BigFloat}(value)
    )[parameter]

struct Dimension

    charge::CentralCharge
    isKac::Bool
    values::Dict{String,Complex{BigFloat}}
    r::Int64;s::Int64

    function Dimension(parameter="",value=0,;Kac=false,r=0,s=0,charge::CentralCharge = CentralCharge("c",1))
        if Kac
            p=.5*(charge["b"]*r - 1/charge["b"]*s)
            new(charge,true, Dict(key => p_to(key,p,charge) for key in ("Δ","δ","P","p")),r,s)
        else
            p=p_from(parameter,value,charge)
            new(charge,true, Dict(key => p_to(key,p,charge) for key in ("Δ","δ","P","p")),r,s)
        end
    end
end

#Print an object of type Dimension
function Base.show(io::IO,dim::Dimension)
    for (key,value) in dim.values
        println(io, "$key = $value")
    end
    if dim.isKac
        println("r = ",dim.r," ; s = ",dim.s)
    end
end

In [25]:
if run_tests
    charge=CentralCharge("c",0)
    println(charge)
    println(Dimension(Kac=true,r=-2.0,s=-1,charge=charge))
end

B = -0.6666666666666666666666666666666666666666666666666666666666666666666666666666695 + 0.0im
c = 0.0 + 0.0im
b = 0.0 + 0.8164965809277260327324280249019637973219824935522233761442308557503201258191083im
β = -0.8164965809277260327324280249019637973219824935522233761442308557503201258191083 + 0.0im

Δ = -2.083333333333333333333333333333333333333333333333333333333333333333333333333322 + 0.0im
P = 1.428869016623520557281749043578436645313469363716390908252403997563060220183424 + 0.0im


δ = -2.041666666666666666666666666666666666666666666666666666666666666666666666666644 - 0.0im
p = 0.0 - 1.428869016623520557281749043578436645313469363716390908252403997563060220183424im
r = -2 ; s = -1

